In [49]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import pandas as pd
import cv2
import torch.optim as optim
from tqdm import tqdm

## preprocessing

In [9]:
train=[]
for i in range(1, 50000 + 1):
    image_path = os.path.join('train', f"{i}.png")
    train.append(image_path)

In [59]:
# Read the CSV file containing labels
labels = pd.read_csv("trainLabels.csv")

# Convert labels to numeric values
labels['numeric_labels'] = pd.factorize(labels['label'])[0]

# Display the first 5 rows
print(labels.head())

main_labels=torch.tensor(labels['numeric_labels'])

print(main_labels[:5])

   id       label  numeric_labels
0   1        frog               0
1   2       truck               1
2   3       truck               1
3   4        deer               2
4   5  automobile               3
tensor([0, 1, 1, 2, 3])


In [25]:
train_images = []
for i in train:
    im = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    if im is not None:
        im = torch.tensor(im, dtype=torch.float32)  
        train_images.append(im)

In [31]:
train_images = torch.cat([img.unsqueeze(0) for img in train_images], dim=0)


In [32]:
print(main_labels[:5])
print(train_images.shape)

tensor([0, 1, 1, 2, 3])
torch.Size([50000, 32, 32])


In [44]:
model=nn.Sequential(
    nn.Conv2d(1,8,3,1,0),#1->no of input, 8-> kernals, 3-> kernal size, 1->stride,  0->padding
    nn.MaxPool2d(2,2),#kernal size , stride
    nn.Flatten(),
     nn.Linear(8 * 15 * 15, 10),  # Adjust the size according to the output shape after flattening
    nn.Softmax(dim=1)  # Softmax activation function
)

In [48]:
# Set the optimizer 
optimizer = optim.SGD(model.parameters(), lr=0.005)

criterion = nn.CrossEntropyLoss()

In [61]:
for epoch in range(3):
    for i in range(len(train_images)):
        inp = train_images[i].unsqueeze(0).unsqueeze(0)  # Adding an extra dimension for batch
        model.train()
        optimizer.zero_grad()
        outs = model(inp)  # Model output
        loss = criterion(outs.squeeze(0), main_labels[i])  # Ensure correct shape for labels
        loss.backward()
        optimizer.step()
    
    # Calculate accuracy
    _, predicted = torch.max(outs, 1)  # `outs` not `outputs` here
    correct = (predicted == main_labels).sum().item()
    accuracy = correct / len(main_labels) * 100

    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}%")


Epoch 1, Loss: 2.454777956008911, Accuracy: 10.0%
Epoch 2, Loss: 2.45932936668396, Accuracy: 10.0%
Epoch 3, Loss: 2.4610745906829834, Accuracy: 10.0%
